In [84]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import numpy as np

In [85]:
links = ['http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/0f69cde4-567f-468b-8bc1-fc861ac26590/download/bdg_serie_iscritti.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/3098c012-08de-4085-b532-66c00e72a6cf/download/bdg_serie_laureati.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/74fc14c9-571f-4181-888e-ef17e014793e/download/bdg_serie_dottorandi.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/155998a7-c183-4eb3-b8cd-ca4d7db72b64/download/bdg_serie_dottori.csv']

In [86]:
def leggi_csv(link):
    df = pd.read_csv(link,encoding = "ISO-8859-1",sep=';')
    df = df.rename(columns={
        df.columns[-1]:'Numero'
    })
    categoria = os.path.split(link)[-1][:-4].split('_')[-1]
    df['categoria'] = [categoria]*len(df)
    return df

In [87]:
dfs = [leggi_csv(x) for x in links]

In [88]:
stud = pd.concat(dfs,join='inner')

In [89]:
stud = stud[stud['CorsoTIPO']!='Vecchio Ordinamento'] #leviamo i vecchi ordinamenti

In [90]:
stud['ANNO'].unique() #uniformiamo tutto sull'anno a sinistra in quelli con formato x/x+1

array(['2021/2022', '2020/2021', '2019/2020', '2018/2019', '2017/2018',
       '2016/2017', '2015/2016', '2014/2015', '2013/2014', '2012/2013',
       2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012],
      dtype=object)

In [91]:
diz = dict(zip(stud['ANNO'].unique()[0:10],stud['ANNO'].unique()[10:])) #mappiamo vecchi valori a nuovi

In [92]:
stud['ANNO'].loc[:] = stud['ANNO'].apply(lambda x: diz.get(x, x)).astype(int)

In [93]:
prof = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/92f2008d-958f-4e9c-ae5c-7a3dd418cd57/download/bdg_serie_academic_staff_ambito.csv',encoding = "ISO-8859-1", delimiter=';')

In [94]:
stud.columns

Index(['ANNO', 'AteneoCOD', 'AteneoNOME', 'AteneoREGIONE', 'AteneoAREAGEO',
       'CorsoTIPO', 'COD_FoET2013', 'DESC_FoET2013', 'Genere', 'Numero',
       'categoria'],
      dtype='object')

In [95]:
prof = prof.rename(columns={
    'CODICE_ATENEO':'AteneoCOD',
    'NOME_ATENEO':'AteneoNOME',
    'REGIONE':'AteneoREGIONE',
    'AREA_GEO': 'AteneoAREAGEO',
    'GENERE':'Genere',
    'N_AcStaff':'Numero',
    'GRADE':'categoria'
})

In [96]:
decod = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/17b34084-9c01-4e90-9260-982fcb982e6a/download/cod_grade.csv', encoding = "ISO-8859-1", delimiter=';')

In [97]:
decod_grade = dict(zip(decod['GRADE'],decod['Qualifica']))

In [98]:
decod_grade['C'] = 'Ricercatore'

In [99]:
prof['categoria'] = prof['categoria'].apply(lambda x: decod_grade[x])

In [100]:
#troviamo per materie prof, se è stem o no!
materie_prof = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/688a0ad5-a4f7-469d-af3d-f7448aa5ae30/download/cod_ford.csv',encoding = "ISO-8859-1", delimiter=';')
decoder_prof = dict(zip(materie_prof['FoRD'],materie_prof['Area STEM']))

In [101]:
prof['STEM'] = prof['FoRD'].apply(lambda x: decoder_prof[x])

In [102]:
#ora stessa cosa per stud
materie_stud = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/3f52db2f-24ce-4605-8e51-5618cc4ff4e3/download/cod_foet2013.csv',encoding = "ISO-8859-1", delimiter=';')
decoder_stud = dict(zip(materie_stud['ISCED_F_1dgt'],materie_stud['Area STEM']))

In [103]:
stud.sample()

,ANNO,AteneoCOD,AteneoNOME,AteneoREGIONE,AteneoAREAGEO,CorsoTIPO,COD_FoET2013,DESC_FoET2013,Genere,Numero,categoria
31515,2012,01001,Genova - Università degli studi,Liguria,NORD-OVEST,Laurea,4,"Business, administration and law",F,1583,iscritti


In [104]:
stud['STEM'] = stud['COD_FoET2013'].apply(lambda x: decoder_stud[x])

In [105]:
df = pd.concat([stud,prof],join='inner') #adesso sappiamo se sono STEM o no!

In [106]:
df['STEM'] = df['STEM'].replace('No',False).replace('Sì',True).astype(bool) #trasformiamo in booleani

In [107]:
def conto(df, verbose=False, relative=False, flourish=False): 
    '''
    
    
    
    big funzione che serve a fare i conti
    
    
    '''
    
    df = df.groupby(['categoria','Genere']).agg({
        'Numero':pd.Series.sum
    }).reset_index()
    
    diz = {
            'iscritti':0,
            'laureati':1,
            'dottorandi':2,
            'dottori':3,
            'Assegnista di Ricerca':4,
            'Ricercatore':5,
            'Prof. Associato':6,
            'Prof. Ordinario':7
        }
    
    def ordina(colonna): #funzione di supporto solo per averli in ordine 
        colonna = colonna.apply(lambda x:diz[x])
        return colonna
    
    df = df.sort_values(by=['categoria'],key=ordina)
    
    if flourish:
        return df
    
    M= []
    F=[]
    colons = []

    for cat in diz:
        try:
            M.append(df[(df['categoria']==cat)&(df['Genere']=='M')]['Numero'].iloc[0])
        except IndexError as e:
            if verbose:
                print(f'M - {cat} ---> {e}')
            M.append(0)
        
        try:
            F.append(df[(df['categoria']==cat)&(df['Genere']=='F')]['Numero'].iloc[0])
            
        except IndexError as e:
            if verbose:
                print(f'F - {cat} ---> {e}')
            F.append(0)
            
        colons.append(cat)
    
    ndf = pd.DataFrame([M,F],columns=colons,index=['M','F'])
    if relative:
        ndf = ndf.apply(lambda x: x/x.sum()*100)
        
    return ndf

pd.DataFrame.conto = conto

In [108]:
def filtro(df, nome, stem=None, anno=2021): 
    '''
    
    funzione per filtrare per università, STEM (False,True,None), anno
    
    
    '''
    
    if stem is not None:
        df = df[df['STEM']==stem]
        
    return df[(df['AteneoNOME']==nome)&(df['ANNO']==anno)]
pd.DataFrame.filtro = filtro

In [109]:
def grafico(df,transpose=True,tit=None, relative=True):
    
    '''
    
    funzione per fare il grafichetto
    
    '''
    if relative:
        df = df.apply(lambda x: x/x.sum()*100)
    
    if transpose:
        df = df.transpose()
    
    fig,ax = plt.subplots(figsize=(14,6))
    plt.ylim(0,100)
    plt.plot(df['M'],'o-', label='maschi')
    plt.plot(df['F'],'o-',label='femmine')
    plt.legend()
    
    if tit:
        ax.set_title(tit)

    for x,y_m,y_f in zip(df.index,df['M'],df['F']):
        plt.annotate(f'{round(y_m,1)}%', # this is the text
                         (x,y_m), # these are the coordinates to position the label
                         textcoords="offset points", # how to position the text
                         xytext=(0,8), # distance from text to points (x,y)
                         ha='center',
                         size=10,
                         style='italic')

        plt.annotate(f'{round(y_f,1)}%', # this is the text
                         (x,y_f), # these are the coordinates to position the label
                         textcoords="offset points", # how to position the text
                         xytext=(0,-13), # distance from text to points (x,y)
                         ha='center',
                         size=10,
                         style='italic')
    return fig

pd.DataFrame.grafico = grafico

In [110]:
def indice(df, considera_studenti=False):
    '''
    Glass Ceiling Index definito dall'UE
    
    1 = donne hanno le stesse chance degli uomini di essere promosse
    0 = donne sono più rappresentate al grado massimo rispetto a uomini
    >1 = uomini più rappresentati
    
    ritorna una tupla con: average, top, indice
    
    '''
    rep_tot_femm = df.loc['F'].iloc[0 if considera_studenti else 5:].sum()
    rep_tot = rep_tot_femm + df.loc['M'].iloc[0 if considera_studenti else 5:].sum()
    rep_ceil_femm = df.loc['F'].iloc[-1]
    rep_ceil = rep_ceil_femm + df.loc['M'].iloc[-1]
    
    avg = rep_tot_femm/rep_tot
    top = rep_ceil_femm/rep_ceil
    
    i = (rep_tot_femm/rep_tot) / (rep_ceil_femm/rep_ceil)
    return avg, top, i

pd.DataFrame.indice = indice

l = []
for ateneo in df['AteneoNOME'].unique():
    ndf = df.filtro(ateneo, stem=True).conto()
    rap = ndf['iscritti']['M']/(ndf['iscritti']['M']+ndf['iscritti']['F'])*100
    l.append((ateneo,rap))

for ateneo in df['AteneoNOME'].unique(): #ci abbiamo fatto tutte le cartelle!
    
    ndf = df.filtro(ateneo, stem=False).conto()
    if 'iscritti' in ndf.columns and ndf['iscritti'].sum() > 15000:
        path = f'/Users/sava/Desktop/Università genere/{ateneo}/NO_STEM'
        os.makedirs(path,exist_ok=True)
        ndf.to_csv(f'{path}/no_stem.csv')
        fig = ndf.grafico(tit=f'{ateneo} - no stem')
        fig.savefig(f'{path}/no_stem.png')
    
    ndf = df.filtro(ateneo, stem=True).conto()
    
    if 'iscritti' in ndf.columns and ndf['iscritti'].sum() > 15000:
        path = f'/Users/sava/Desktop/Università genere/{ateneo}/STEM'
        os.makedirs(path,exist_ok=True)
        ndf.to_csv(f'{path}/stem.csv')
        ndf.grafico(tit=f'{ateneo} - stem')
        fig = ndf.grafico(tit=f'{ateneo} - no stem')
        fig.savefig(f'{path}/stem.png')

In [111]:
df['AteneoNOME'].unique()

array(['TOTALE ATENEI', 'Cagliari - Università degli studi',
       'Sassari - Università degli studi',
       'Catania - Università degli studi',
       'Enna - Libera Università della Sicilia Centrale "KORE"',
       'Messina - Università degli studi',
       'Palermo - Università degli studi',
       'Reggio Calabria - Università per Stranieri',
       'Reggio Calabria - Università degli studi Mediterranea',
       'Catanzaro - Università degli studi "Magna Grecia"',
       'Arcavacata di Rende - Università della Calabria',
       'Potenza - Università degli studi della Basilicata',
       'Lecce - Università del Salento',
       'Casamassima - Libera Università Mediterranea "Giuseppe Degennaro"',
       'Bari - Politecnico', 'Bari - Università degli studi',
       'Foggia - Università degli studi',
       'Campobasso - Università degli studi del Molise',
       'Torrevecchia Teatina (CH) - Università telematica "Leonardo da Vinci"',
       "Chieti e Pescara - Università degli studi

In [112]:
unis=[]
avgs = []
tops = []
inds = []
cat = []
for uni in df['AteneoNOME'].unique():
    avg,top,ind = df.filtro(uni,anno=2021).conto().indice(considera_studenti=True)
    avgs.append(avg)
    tops.append(top)
    inds.append(ind)
    unis.append(uni)
    cat.append('Da iscritte a professoresse ordinarie')
    
    avg,top,ind = df.filtro(uni,anno=2021).conto().indice(considera_studenti=False)
    avgs.append(avg)
    tops.append(top)
    inds.append(ind)
    unis.append(uni)
    cat.append('Da ricercatrici a professoresse ordinarie')

/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in long_scalars
/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars
/Users/sava/opt/anaconda3/envs/pythonforjournalists/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


In [119]:
ndf = pd.DataFrame({
    'uni':unis,
    'medie':avgs,
    'top':tops,
    'ind':inds,
    'cat':cat
})
ndf = ndf.dropna().sort_values(by=['cat','ind'])

In [120]:
ndf = ndf[ndf['ind']!=np.inf]

In [121]:
ndf['medie']=ndf['medie']*100
ndf['top']=ndf['top']*100

In [122]:
ndf

,uni,medie,top,ind,cat
168,"Novedrate (CO) - Università telematica ""e-Campus""",53.998512,60.000000,0.899975,Da iscritte a professoresse ordinarie
136,Venezia - Università IUAV,58.046694,41.818182,1.388073,Da iscritte a professoresse ordinarie
100,Perugia - Università per stranieri,65.247148,46.153846,1.413688,Da iscritte a professoresse ordinarie
176,Aosta - Università degli studi,70.685579,50.000000,1.413712,Da iscritte a professoresse ordinarie
164,Milano - Politecnico,34.690729,23.809524,1.457011,Da iscritte a professoresse ordinarie
...,...,...,...,...,...
203,Roma - Universitas Mercatorum,44.444444,18.181818,2.444444,Da ricercatrici a professoresse ordinarie
69,"Roma - Università telematica ""San Raffaele"" - ...",58.928571,23.076923,2.553571,Da ricercatrici a professoresse ordinarie
67,Roma - Link Campus University,45.833333,16.666667,2.750000,Da ricercatrici a professoresse ordinarie
79,"Roma - Università telematica ""Unitelma Sapienza""",50.000000,18.181818,2.750000,Da ricercatrici a professoresse ordinarie


In [123]:
ndf.to_csv('GCI.csv',index=False)

# per questionario

In [63]:
df.filtro('Roma - Università degli studi "La Sapienza"').conto().indice()

1.444780089294196

In [64]:
df.filtro('Palermo - Università degli studi').conto().indice()

1.4924130566614813

In [65]:
df.filtro('Napoli - Università degli studi "Federico II"').conto().indice()

1.5401583842297377

In [66]:
df.filtro('Milano - Università degli studi').conto().indice()

1.416000679600865

In [69]:
ndf[ndf['uni'].str.contains('torino',flags=re.I)]

,uni,ind
92,Torino - Università degli studi,1.362921
91,Torino - Politecnico,1.618755
